## Step2: Specialization Similarity Ranking
## Step3: Matched Control Researcher

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None
import pickle
from tqdm import tqdm
import seaborn as sns
import os, sys, gc, random, itertools, math, pickle
from itertools import combinations
import networkx as nx


In [2]:
# Read Author IDs of the guideline members
authid_12 = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/2021/A1.2_who_guideline_development_group.npy')
authid_13 = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/A1.3_who_systematic_review_team_revise.npy')
authid_14 = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/2021/A1.4_who_external_methodologists.npy')
authid_15 = np.load('//disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/A1.5_who_external_review_team_revise.npy')

boradmember_authid = list(authid_12)+list(authid_13)+list(authid_14)+list(authid_15)

authid_list1 = [35780677200 if x == 7006806977 else 57206422848 if x == 54791745100  else 57219144066 if x == 55588756600  else 57219144066 if x == 57191727610  else 57219144066 if x == 14037485900 else x for x in boradmember_authid]
treatment_authids = list(set(authid_list1))

In [3]:
candidate_feature_value = pd.read_pickle('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/all_guideline_matching/candidate_feature_value.pickle')

In [4]:
candidate_feature_value

,paper_num_2015,academic_citation_2011,academic_citation_2012,academic_citation_2013,academic_citation_2014,academic_citation_2015,academic_citation_2016,academic_citation_2017,academic_citation_2018,academic_citation_2019,...,policy_citation_2014,policy_citation_2015,policy_citation_2016,policy_citation_2017,policy_citation_2018,policy_citation_2019,policy_citation_2020,policy_citation_2021,policy_citation_2022,policy_citation_2023
authids,,,,,,,,,,,,,,,,,,,,,
11140076200,38.0,836.0,1017.0,1196.0,1360.0,1580.0,1831.0,2080.0,2407.0,2746.0,...,4.0,4.0,7.0,8.0,11.0,12.0,15.0,17.0,17.0,19.0
8858824900,32.0,168.0,221.0,287.0,335.0,407.0,480.0,572.0,663.0,807.0,...,0.0,0.0,0.0,0.0,2.0,9.0,9.0,10.0,12.0,12.0
55835033800,4.0,0.0,0.0,1.0,10.0,20.0,40.0,57.0,76.0,91.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21743832000,27.0,172.0,243.0,314.0,365.0,451.0,511.0,564.0,623.0,666.0,...,15.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0
36642045300,50.0,120.0,260.0,399.0,565.0,756.0,1063.0,1361.0,1686.0,2075.0,...,4.0,10.0,10.0,11.0,20.0,24.0,35.0,49.0,55.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23487676500,114.0,1316.0,1586.0,1918.0,2439.0,3107.0,4067.0,5075.0,6527.0,8262.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
23487806800,53.0,240.0,372.0,534.0,734.0,997.0,1380.0,1738.0,2112.0,2483.0,...,16.0,16.0,20.0,20.0,25.0,29.0,36.0,44.0,48.0,49.0
56371163300,93.0,908.0,1130.0,1367.0,1657.0,1972.0,2306.0,2693.0,3141.0,3552.0,...,7.0,9.0,10.0,15.0,17.0,25.0,35.0,44.0,57.0,74.0


In [5]:
top_k_similar_authids = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/all_guideline_matching/top_k_similar_authids.npy', allow_pickle=True)
top_k_similar_authids = top_k_similar_authids.item()

In [6]:
# Read top_k_similar_authids and remove treatment_authids
new_top_k_similar_authids_dict = {}

for index, top_similar_authid in zip(top_k_similar_authids.keys(), top_k_similar_authids.values()):
    tmp = []
    for authid in top_similar_authid:
        if authid not in treatment_authids:
            tmp.append(authid)
    new_top_k_similar_authids_dict[index] = tmp
            

In [7]:
## Focus on the top 20 similar authors for each author
top_k_similar_authids_dict = {}

for index, top_similar_authid in zip(new_top_k_similar_authids_dict.keys(), new_top_k_similar_authids_dict.values()):
    tmp = []
    for authid in top_similar_authid:
        tmp.append(authid)
    top_k_similar_authids_dict[index] = tmp[:20]

In [8]:
candidate_feature_value.head()

,paper_num_2015,academic_citation_2011,academic_citation_2012,academic_citation_2013,academic_citation_2014,academic_citation_2015,academic_citation_2016,academic_citation_2017,academic_citation_2018,academic_citation_2019,...,policy_citation_2014,policy_citation_2015,policy_citation_2016,policy_citation_2017,policy_citation_2018,policy_citation_2019,policy_citation_2020,policy_citation_2021,policy_citation_2022,policy_citation_2023
authids,,,,,,,,,,,,,,,,,,,,,
11140076200,38.0,836.0,1017.0,1196.0,1360.0,1580.0,1831.0,2080.0,2407.0,2746.0,...,4.0,4.0,7.0,8.0,11.0,12.0,15.0,17.0,17.0,19.0
8858824900,32.0,168.0,221.0,287.0,335.0,407.0,480.0,572.0,663.0,807.0,...,0.0,0.0,0.0,0.0,2.0,9.0,9.0,10.0,12.0,12.0
55835033800,4.0,0.0,0.0,1.0,10.0,20.0,40.0,57.0,76.0,91.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21743832000,27.0,172.0,243.0,314.0,365.0,451.0,511.0,564.0,623.0,666.0,...,15.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0
36642045300,50.0,120.0,260.0,399.0,565.0,756.0,1063.0,1361.0,1686.0,2075.0,...,4.0,10.0,10.0,11.0,20.0,24.0,35.0,49.0,55.0,60.0


In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist

# Extracting guideline researchers
guideline_researchers = treatment_authids

# List to store the matched control candidates
matched_control_candidate_group = {}

matched_control_group = []

used_treatment_authids = []

for guideline_researcher in tqdm(treatment_authids):

    if guideline_researcher in top_k_similar_authids_dict.keys():

        used_treatment_authids.append(guideline_researcher)
    
        top_n_candidate_authid = top_k_similar_authids_dict[guideline_researcher]
        candidate_researcher = list(top_n_candidate_authid)+list([guideline_researcher])
    
        # Create a DataFrame with the candidates
        yearly_num_cited_df = candidate_feature_value[candidate_feature_value.index.isin(candidate_researcher)]
        treat_citation_history = yearly_num_cited_df[yearly_num_cited_df.index==guideline_researcher][[f'academic_citation_{i}' for i in range(2011,2016)]+['policy_citation_2015','paper_num_2015']]
        treat_citation_history = treat_citation_history.astype(float)
        
        control_candidate_citation_history = yearly_num_cited_df.loc[top_n_candidate_authid][[f'academic_citation_{i}' for i in range(2011,2016)]+['policy_citation_2015','paper_num_2015']]
        control_candidate_citation_history = control_candidate_citation_history.astype(float)
        
        treat_citation_history_series = treat_citation_history.loc[guideline_researcher].tolist()
        diff = control_candidate_citation_history.sub(treat_citation_history_series, axis='columns') #academic_citation_2011-2015とpolicy_citation_2015で平均二乗誤差
        mse = np.sqrt((diff**2).sum(axis=1))
        
        # Sort the candidates based on the MSE
        n = 10
        similar = mse.sort_values(ascending=True)[0:n].index.values
    
        for index, candidate in enumerate(list(similar)):
            if candidate not in matched_control_group:
                matched_control_group.append(candidate)
                break
                
            
      
        #plt.plot(yearly_num_cited_df.loc[guideline_researcher,['academic_citation_2011','academic_citation_2012','academic_citation_2013','academic_citation_2014','academic_citation_2015','policy_citation_2015','paper_num_2015']], color="r", linewidth=5,alpha=0.5, label="Nobel Laureates")
        
        #for i in list(similar):
            #plt.plot(yearly_num_cited_df.loc[i,['academic_citation_2011','academic_citation_2012','academic_citation_2013','academic_citation_2014','academic_citation_2015','policy_citation_2015','paper_num_2015']], color="b",alpha=0.5)
        #plt.plot(yearly_num_cited_df.loc[list(similar)[index],['academic_citation_2011','academic_citation_2012','academic_citation_2013','academic_citation_2014','academic_citation_2015','policy_citation_2015','paper_num_2015']], color="b",alpha=0.5,linewidth=5, label=f"top of {n} similar Researchers")
        
        #plt.xticks(rotation=45)
        #plt.legend()
        #plt.show()
    else:
        continue


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 108/108 [00:00<00:00, 262.90it/s]


In [11]:
yearly_num_cited_df.head()

,paper_num_2015,academic_citation_2011,academic_citation_2012,academic_citation_2013,academic_citation_2014,academic_citation_2015,academic_citation_2016,academic_citation_2017,academic_citation_2018,academic_citation_2019,...,policy_citation_2014,policy_citation_2015,policy_citation_2016,policy_citation_2017,policy_citation_2018,policy_citation_2019,policy_citation_2020,policy_citation_2021,policy_citation_2022,policy_citation_2023
authids,,,,,,,,,,,,,,,,,,,,,
8858824900,32.0,168.0,221.0,287.0,335.0,407.0,480.0,572.0,663.0,807.0,...,0.0,0.0,0.0,0.0,2.0,9.0,9.0,10.0,12.0,12.0
22836266000,24.0,142.0,189.0,234.0,297.0,362.0,468.0,603.0,745.0,883.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
7003378870,72.0,1773.0,2142.0,2502.0,2832.0,3195.0,3537.0,3893.0,4266.0,4589.0,...,3.0,6.0,7.0,7.0,7.0,9.0,9.0,9.0,9.0,9.0
7003568845,192.0,1588.0,2041.0,2469.0,2955.0,3453.0,3957.0,4511.0,5080.0,5671.0,...,1.0,6.0,9.0,9.0,9.0,15.0,16.0,17.0,17.0,18.0
7003711284,182.0,2532.0,3038.0,3549.0,4025.0,4530.0,4972.0,5428.0,5909.0,6372.0,...,12.0,16.0,17.0,18.0,20.0,23.0,23.0,28.0,28.0,28.0


In [12]:
# Create a DataFrame with the matched treatment and control groups
matched_data = pd.DataFrame()

treatment_data = candidate_feature_value[candidate_feature_value.index.isin(used_treatment_authids)]
control_data = candidate_feature_value[candidate_feature_value.index.isin(matched_control_group)]

matched_data = pd.concat([treatment_data, control_data])

matched_data['guideline_member'] = 0
matched_data.loc[matched_data.index.isin(used_treatment_authids), 'guideline_member'] = 1

In [13]:
matched_data.head()

,paper_num_2015,academic_citation_2011,academic_citation_2012,academic_citation_2013,academic_citation_2014,academic_citation_2015,academic_citation_2016,academic_citation_2017,academic_citation_2018,academic_citation_2019,...,policy_citation_2015,policy_citation_2016,policy_citation_2017,policy_citation_2018,policy_citation_2019,policy_citation_2020,policy_citation_2021,policy_citation_2022,policy_citation_2023,guideline_member
authids,,,,,,,,,,,,,,,,,,,,,
35168055500,23.0,108.0,165.0,236.0,326.0,490.0,666.0,829.0,1049.0,1330.0,...,1.0,2.0,3.0,3.0,5.0,6.0,6.0,11.0,12.0,1
8729078200,112.0,1621.0,2235.0,2850.0,3593.0,4481.0,5611.0,6924.0,8717.0,10609.0,...,8.0,17.0,20.0,20.0,20.0,23.0,35.0,48.0,56.0,1
10339863000,21.0,110.0,151.0,197.0,256.0,320.0,400.0,508.0,625.0,750.0,...,1.0,2.0,2.0,3.0,3.0,3.0,4.0,5.0,5.0,1
55437181900,85.0,3110.0,3475.0,3896.0,4352.0,4840.0,5393.0,5974.0,6594.0,7217.0,...,57.0,69.0,73.0,78.0,94.0,100.0,131.0,134.0,136.0,1
35579376100,35.0,153.0,190.0,230.0,272.0,316.0,367.0,452.0,550.0,668.0,...,1.0,1.0,3.0,5.0,6.0,9.0,10.0,10.0,11.0,1


In [105]:
# matched_data.to_pickle('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/all_guideline_matching/matched_data.pickle')